In [ ]:
import numpy as np
import cv2
import time
from tkinter import *
from IPython.display import clear_output


cap = cv2.VideoCapture(0)
ret, frame = cap.read ()
sh = frame.shape

cv2.namedWindow ("result", cv2.WINDOW_NORMAL)

#width = 960
#height = 720

width = sh [1]
height = sh[0]

kletka_x = 8
kletka_y = 8
dev1 = width//kletka_x

dev2 = height//kletka_y


cv2.resizeWindow ("result", (width, height))

hsv_min = np.array((53, 55, 147), np.uint8)
hsv_max = np.array((83, 160, 255), np.uint8) 

up = [0, 0]
right = [0, 0]
left = [0, 0]
down = [0, 0]

x_array = []
y_array = []

def measurments(x_array, y_array):
    ###find top position###
    top = [0, 0]
    for i in range(len(y_array)):
        if y_array[i] > top[1]:
            top[0] = x_array[i]
            top[1] = y_array[i]
    ###find left position###
    left = [0, 0]
    left[0] = 10000
    for i in range(len(x_array)):
        if x_array[i] < left[0]:
            left[0] = x_array[i]
            left[1] = y_array[i]
    ###find right position###
    right = [0, 0]
    right[0] = 0
    for i in range(len(x_array)):
        if x_array[i] > right[0]:
            right[0] = x_array[i]
            right[1] = y_array[i]
    ###find botom position###
    botom = [0, 0]
    botom[1] = 10000
    for i in range(len(y_array)):
        if y_array[i] < botom[1]:
            botom[0] = x_array[i]
            botom[1] = y_array[i]
            
    return top, left, right, botom

while True:
    flag, img = cap.read()
    # преобразуем RGB картинку в HSV модель
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV )
    # применяем цветовой фильтр
    thresh = cv2.inRange(hsv, hsv_min, hsv_max)

    # вычисляем моменты изображения
    moments = cv2.moments(thresh, 1)
    dM01 = moments['m01']
    dM10 = moments['m10']
    dArea = moments['m00']
    # будем реагировать только на те моменты,
    # которые содержать больше 100 пикселей

    keyb = cv2.waitKey(1)
    
    #если нужно увидеть сетку всего экрана
    
    i = 0
    j = 0   
    while i <= img.shape[1]:#width:
        while j <= img.shape[0]:#height:
            cv2.rectangle(img,(i, j),(i+dev1, j+dev2),(255,255,255),1)
            j+= dev2
        i+= dev1
        j = 0 
    
    cv2.rectangle(img,(left[0], up[1]),(right[0]+dev1, down[1]+dev2),(255,0,255),3)
    
    if dArea > 100:
        x = int(dM10 / dArea)
        y = int(dM01 / dArea)
        cv2.circle(img, (x, y), 10, (0,0,255), -1)
        
        k_i = 0
        l_j = 0
        while k_i <= img.shape[1]:
            while l_j <= img.shape[0]:
                if x < k_i + dev1 and x > k_i and y < l_j + dev2 and y > l_j:
                    cv2.rectangle(img,(k_i, l_j),(k_i+dev1, l_j+dev2),(0,255,255),3)
                    
                    if up == [0, 0] or right == [0, 0] or left == [0, 0] or down == [0, 0]:
                        x_array.append(k_i)
                        y_array.append(l_j)
                        if (keyb & 0xFF == ord ('c')):
                            up, left, right, down = measurments(x_array, y_array)       
                        
                l_j+= dev2
            k_i+= dev1
            l_j = 0
                
    cv2.imshow('result', img)
    
    if (cv2.waitKey(1) & 0xFF == ord ('q')):
          break

cap.release()
cv2.destroyAllWindows()